In [2]:
'''
The code is used to train BC imitator, or pretrained GAIL imitator
'''

import argparse
import tempfile
import os.path as osp
import gym
import logging
from tqdm import tqdm

import tensorflow as tf

from baselines.gail import mlp_policy
from baselines import bench
from baselines import logger
from baselines.common import set_global_seeds, tf_util as U
from baselines.common.misc_util import boolean_flag

#from baselines.gail.run_mujoco import runner
from baselines.gail.dataset.mujoco_dset import Mujoco_Dset 

##from baselines.common.mpi_adam import MpiAdam

In [3]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment
network = "gfootball_impala_cnn"
scenario = "../_scenarios/academy/pass_n_shoot.scenic"
load_path = "/home/ubuntu/ScenicGFootBall/training/gfrl/_saved_models/pass_shoot_5M"



In [4]:


import tensorflow.compat.v1 as tf
import multiprocessing

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])



ncpu = multiprocessing.cpu_count()
config = tf.ConfigProto(allow_soft_placement=True,
                        intra_op_parallelism_threads=ncpu,
                        inter_op_parallelism_threads=ncpu)
config.gpu_options.allow_growth = True
tf.Session(config=config).__enter__()

from gfrl.common.mybase import ppo2
model = ppo2.learn(
    network=network,
    total_timesteps=0,
    env=vec_env,
    nsteps=512,
    load_path=load_path
    )




/home/ubuntu/.local/lib/python3.7/site-packages/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
Logging to /tmp/openai-2021-04-27-07-17-25-173456





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [16]:
from gfrl.dm.my_runner import MyTrajRunner

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])

num_episodes = 10
runner = MyTrajRunner(env=vec_env, model=model, nsteps=512, num_episodes=num_episodes)
mb_obs, mb_rewards, mb_dones, mb_actions, mb_values, mb_neglogpacs, mb_states, epinfos, mb_gt = runner.run()

mb_act_oh = np.zeros((mb_actions.shape[0], mb_actions.max()+1))
mb_act_oh[np.arange(mb_actions.shape[0]), mb_actions] = 1

Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
in run


In [25]:
np.sum(mb_dones)

9

In [17]:
import numpy as np
np.sum(mb_rewards)

10.0

In [18]:
print(mb_obs.shape, mb_actions.shape)

(286, 72, 96, 16) (286,)


In [19]:
mb_actions.reshape(-1,1).shape

(286, 1)

In [22]:
mb_act_oh.shape

(286, 19)

In [23]:
mb_actions[0:5]

array([9, 9, 9, 9, 7])

In [24]:
mb_act_oh[0:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]])

In [26]:
expert_file_name = f"../_data/pns_{num_episodes}.npz"
import numpy as np 
np.savez(expert_file_name, obs=mb_obs, acs=mb_act_oh, num_epi = num_episodes, mean_reward = np.sum(mb_rewards)/num_episodes)

In [13]:

#expert_file_name = "../_data/pns_100.npz"
#import numpy as np 
#np.savez(expert_file_name, obs=mb_obs, acs=mb_actions)


In [27]:
td = np.load(expert_file_name)

In [28]:
list(td.keys())

['obs', 'acs', 'num_epi', 'mean_reward']

In [13]:
dataset = GFDset(expert_file_name)

In [30]:
def generate_expert_data_with_rewards(env, num_interactions=1000, file_name="expert_data"):
    expert_observations = []
    expert_actions = []
    expert_rewards = []

        
    obs = env.reset()
    tr = 0
    for i in tqdm(range(num_interactions)):
        expert_observations.append(obs)

        obs, reward, done, info = env.step(env.action_space.sample())
        tr+=reward
        # print(info)
        action = info["action_taken"]
        expert_actions.append(action)

        if done:
            obs = env.reset()
            expert_rewards.append(tr)
            tr = 0

    expert_observations = np.array(expert_observations)
    expert_observations = np.moveaxis(expert_observations, [3], [1])
    expert_actions = np.array(expert_actions)
    expert_rewards = np.array(expert_rewards)
    print("Expert observation shape: ", expert_observations.shape)
    print("Expert actions shape: ", expert_actions.shape)

    np.savez_compressed(
        file_name,
        expert_actions=expert_actions,
        expert_observations=expert_observations,
        expert_rewards=expert_rewards
    )
    return expert_observations, expert_actions, expert_rewards